In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
import os

In [ ]:
# Filepath for the dataset
FILE_PATH = "data/BCN-HRB_Points.txt"

In [ ]:
# Parameters
SKIPPED_COL = 10  # Columns before starting the time series data
ANOMALY_PERIOD = 10  # The period used to detect anomalies

In [ ]:
# Data Preparation

def load_data(filepath):
    """Load dataset from file."""
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"File {filepath} not found.")
    
    data = np.loadtxt(filepath, skiprows=1)
    return data

def preprocess_data(data):
    """Preprocess the dataset by splitting into time series, train/test sets."""
    ts_data = data[:, SKIPPED_COL:]  # Time series of all points
    len_ts = data.shape[1]
    
    input_model = data[:, len_ts-70:]  # Last one year values of time series
    input_train_test = data[:, len_ts-131:len_ts-70]

    x = input_train_test[:, :-1]
    y = input_train_test[:, -1:]

    return x, y, input_model, ts_data

def split_data(x, y, test_size=0.3):
    """Split data into training and testing sets."""
    return train_test_split(x, y, test_size=test_size, shuffle=False)

In [ ]:
# Functions to evaluate adjacent TS

def residual_std(time_series):
    """Calculate the residual standard deviation."""
    x = np.arange(0, len(time_series) * 6, 6)
    y = np.polyfit(x, time_series, 3, full=True)
    rstd = np.sqrt(y[1] / (len(time_series) - 2))
    return rstd

def neighbor_ps(input_info, input_ts, n_row, n_col):
    """
    Find the time series of neighboring points based on row and column.
    """
    ts_n = np.empty((0, input_ts.shape[1]), float)

    for i in range(-2, 3):
        for j in range(-7, 8):
            c = np.array([n_row + i, n_col + j])
            id_n = np.argwhere(np.all(input_info[:, 1:3] == c, axis=1))
            
            if len(id_n) > 0 and not np.array_equal(c, [n_row, n_col]):
                psts = input_ts[id_n, :].reshape(id_n.shape[0], -1)
                ts_n = np.vstack((ts_n, psts[0, :]))
                
    return ts_n

In [ ]:
# Machine Learning model

def train_model(X_train, y_train):
    """Train XGBoost model."""
    regressor = MultiOutputRegressor(xgb.XGBRegressor())
    return regressor.fit(X_train, y_train)

In [ ]:
def main():
    """Main function to run anomaly detection and prediction."""
    # Load and preprocess data
    data = load_data(FILE_PATH)
    x, y, input_model, ts_data = preprocess_data(data)

    # Split data
    X_train, X_test, y_train, y_test = split_data(x, y)
    print(f"Train/Test shapes: {X_train.shape}, {X_test.shape}, {y_train.shape}, {y_test.shape}")

    # Train model
    model_xgb = train_model(X_train, y_train)

    # Variables to store results
    alerts_full = np.empty((0, ANOMALY_PERIOD - 1), float)
    alerts = []

    # Process each time series point for anomaly detection
    for j in range(data.shape[0]):
        X_0 = input_model[j:j + 1, :-ANOMALY_PERIOD]
        XP = input_model[j, -ANOMALY_PERIOD:]
        n_row, n_col = data[j, 1], data[j, 2]
        x_n = neighbor_ps(data, input_model, n_row, n_col)
        thr = 1.96 * residual_std(ts_data[j, :-ANOMALY_PERIOD])

        A = []  # Store alert values
        for i in range(0, len(XP) - 1):
            P = model_xgb.predict(X_0)
            n_p = np.mean(x_n[:, input_model.shape[1] - ANOMALY_PERIOD + i]) if len(x_n) > 0 else XP[i + 1]

            # Detect anomalies based on thresholds
            if abs(P - XP[i]) <= thr:
                A.append(0)
            elif abs(P - XP[i]) > thr and abs(P - n_p) <= thr:
                A.append(0)
            elif (P < XP[i] - thr and P > n_p + thr) or (P > XP[i] + thr and P < n_p - thr):
                A.append(0)
            elif (P < XP[i] - thr and P < n_p - thr) or (P > XP[i] + thr and P > n_p + thr):
                A.append(2)

        alerts_full = np.vstack((alerts_full, A))
        alerts.append(sum(A))

    # Save results
    np.savetxt('results/alerts.csv', np.array(alerts).reshape(-1, 1), delimiter=',')
    np.savetxt('results/alerts_full.csv', alerts_full, delimiter=',')

if __name__ == "__main__":
    main()